<h1>Research iteration 1</h1>

<i>Sjoerd Beetsma, Maarten de Jeu
Class V2A - Group 5</i>

<h2>Introduction</h2>

TODO(m-jeu): Place to write a quick introduction to the notebook structure, conventions, etc.

<h2>Business Understanding</h2>

Through the business, we have been asked to examine the following questions:

<ol>
<li>In hoeverre is de score van een Portugese Red te voorspellen op basis van de chemische kenmerken?</li>
<li>Zelf te bepalen: denk aan andere type wijnen of andere landen.</li>
<li>Zelf te bepalen: denk bijvoorbeeld aan het clusteren op basis van de chemische kenmerken waarmee het type druif of de regio bepaald kan worden. Of kun je logische clusters vinden van topwijnen, doordrinkwijnen en bocht?</li>
</ol>

<i>TODO(m-jeu): Either translate these questions to english, or change the language of the rest of the document to dutch. This also goes for the dutch list in 'Data Understanding'.</i>

Currently, the exact nature of 'the business' and their desires datascience-wise is unknown to us. This requires further investigation.

<h2> Data Understanding </h2>

The business tells us the most important variables in the dataset are:

<ol>
<li>Herkomst van de wijn en type druif.</li>
<li>Review van de wijn, inclusief naam van de sommelier en de score op een schaal van 1 tot 100.</li>
<li>De uitkomsten van chemische tests op 11 waarde (waaronder suikergehalte, pH, alcoholgehalte, et cetera).</li>

We import some libraries, and the dataset. Then we have an initial look at it:

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
dataset = pd.read_csv("datasets/redwine.csv", sep=";")
dataset.head(5)

Rows seems to correspond with individual wines on first glance, though this does need to be examined more thoroughly. Columns are different attributes for those individual wines.

<h3>Feature variables</h3>

Here we remove the columns: country, variety, description, title, taster_name and id. They are unnecessary for our model in determining the Target, which is points.
Country and variety are possible feature variables but are left out for now as the dataset contains only 1 unique country and variety.

In [ ]:
dataset_features_target = dataset.drop(['country', 'variety', 'description', 'title', 'taster_name', 'id'], axis=1)
dataset_features_target.head(0)

<h3> Scales of measurements </h3>

To construct an appropiate model it's necessary to have a understanding of all the scales of measurements for the target and feature variables.

In [ ]:
nom, disc, cont = 'Nominal', 'Discrete', 'Continous'
vars_scale = [nom, disc, disc, nom, nom, cont, cont, cont, cont, cont, cont, cont, cont, cont, cont, cont ]
measurement_scales = pd.DataFrame({'Variable':dataset_features_target.columns, 'Scale of measurement':vars_scale})
measurement_scales

<h3>Central tendancies and dispersion measures</h3>

From the central tendancies and dispersion measures we can see some useful statistics about the target and feature variables.

In [ ]:
dataset_features_target.describe().round(2)

To be noted the feature variables alcohol, density, citric acid aren't described even though they are of discrete scale, we will look at what's wrong during the data preparation phase.

<h3>Distribution of data</h3>

Lets take a more visual look at the distribution of all the variables through a histogram for each of the feature and target attributes.

In [ ]:
dataset_features_target.hist(figsize=(15,15), bins = 20)
plt.show()

The distributions of pH seems to follow a normal distribution.
From the points and price histogram we can see that most wines range from a relatively high score between 85 and 90. And most wines ranging in a price from around 5 to 30.

<h3>Outliers</h3>

To get a visual understanding of the outliers in the feature columns each feature gets a boxplotted with the target variable points. Giving a small summary of the minimum, Q1, Q2 (median), Q3 and the maximum of each attribute plotted against points scored to give a view of outliers at all quality/point levels.

In [ ]:
for col in dataset_features_target.select_dtypes(include=np.number, exclude=None).drop('points', axis=1): # x-axis requires numerical value and don't plot id
    sns.boxplot(x=dataset['points'], y=dataset[col])
    plt.show()

As can be seen from the boxplots all of our current variables (beside the ones that need to be cleaned before further data understanding) contain outliers.

All outliers in the above boxplots seem to be plausible and not from incorrect data.
From the boxplot with price on the y axis and points on the x axis we can clearly see that the price of a wine is related to the quality of the wine.
Also we can tell that wines with a quality of 92 or higher have a volatile acidity below 1.

<h3>Correlations</h3>

To help find positive, negative and neutral correlations matrix is constructed where dark red corresponds to a positive correlation and green a negative correlation.

We create this correlation matrix with the help of a function, so that we can reuse it later.

In [ ]:
def correlation_matrix(dataset: pd.DataFrame):
    """Return a correlation matrix created using seaborn and matplotlib that for all columns in
    a pandas dataframe.

    Args:
        dataset: Dataset to construct correlation matrix for.

    Returns:
        correlation matrix."""
    corr = dataset.corr()
    plt.figure(figsize=(10,7.5))
    cmap = sns.diverging_palette(200, 0, as_cmap=True) # color palette as cmap
    mask = np.logical_not(np.tril(np.ones_like(corr))) # triangle mask
    sns.heatmap(corr,annot=True, mask=mask, cmap = cmap, vmin=-1, vmax=1) # correlation heatmap


correlation_matrix(dataset_features_target)

In the correlation matrix graph above you can see which attributes have a correlation to other attributes. Starting with our target variable 'quality', we can see quality has a few correlations with the strongest one being alcohol and a few weaker ones like volatile acidity, sulphates and citric acid. Because quality is our target variable it's independent attribute in the correlation.

Getting rid of outliers might bring the absolute value of the correlation scores closer to one, but this would be something to look at again during data preparation.

Besides there are some correlations among chemical properties:
Fixed acidity has strong correlation with pH, but it’s still an independent type. pH However is a dependent type; it depends on the former. Volatile acidity, residual sugar, sulphates, chlorides, and density are all independent data types. Total sulfur dioxide is dependent on free sulfur dioxide, but free sulfur dioxide is independent.

<h2>Data Preparation</h2>

The data needs some cleaning up. Because it might be useful to be able to view columns that were droppped in Data Exploration to gain more understanding, like wine titles, we'll be operating on the original dataset.

The current policy we'll be adhering to outlier-wise, is that outliers that are somewhat realistic won't be dropped (for now). Our philosophy is that these might still be relevant for some models, so we won't drop them until absolutely necessary. Outliers that are obvious errors will be dropped.

An overview of datatypes:

In [ ]:
dataset.dtypes

Jupyter doesn't recognize some of the Python str objects for what they are, and simply calls them the 'object' type. Let's convert them to the right type to allow for more method flexibility:

In [ ]:
dataset = dataset.convert_dtypes()
dataset.dtypes

There are some datatypes in the DataFrame that don't quite correspond to what you'd expect them to be, considering what they represent. Citric acid, Density and Alcohol are string objects, even though you'd expect them to be some kind of number-datatype. Let's copy everyone to a seperate column called 'raw_columnname' so that we can evaluate the original next to the converted in case we run into any problems turning these columns into numbers. Then we'll convert each one individually.

In [ ]:
problematic_column_names = ["citric acid", "density", "alcohol"]
copied_column_names = {f"raw_{name}":name for name in problematic_column_names}

for new, to_copy in copied_column_names.items():  # Is this allowed?
    dataset[new] = dataset[to_copy]

<h3>Data Preparation on column-by-column basis</h3>

<h4>Citric Acid</h4>

The citric acid column consists of string objects, through most entries are formatted like floats. Pandas can convert these for us, turning the ones that it can't understand into not-a-number entries.

In [ ]:
dataset["citric acid"]

In [ ]:
dataset["citric acid"] = pd.to_numeric(dataset["citric acid"], errors='coerce')

And let's define a simple convenience function that describes the min, max and amount of nan entries in a series object, to quickly gauge the validity of data contained within, and call it on the Citric Acid column.

In [ ]:
def simple_describe(s: pd.Series, name: str = "Series object") -> None:
    """Print out a simple description of a Pandas Series object that contains numeric values.
    That covers min, max and amount of nan/None entries.

    Args:
        s: Pandas series object with numeric values.
        name: optional name for the series to use in the printed description."""
    print(f"{name}:\nmin: {np.min(s)}\nmax: {np.max(s)}\n#(nan): {s.isnull().sum()}")

simple_describe(dataset["citric acid"], "Citric acid")

Auto converting the citric acid table to float leaves us with 203 not-a-number entries. Lets have a look at them.

In [ ]:
dataset.loc[dataset["citric acid"].isnull()][["id", "title", "citric acid", "raw_citric acid"]]

It appears these data points did not have a value for citric acid to begin with. According to <a href="https://wineserver.ucdavis.edu/industry-info/enology/methods-and-techniques/common-chemical-reagents/citric-acid">this website</a>, citric acid is something that's <i>added</i> to wine most of the time, but it doesn't seem essential. We'll assume that the wines with '-' for citric acid were given this value on purpose, and that this means that there's none in there. Let's replace those values with 0 in the raw table, and use that to create a final version of the citric acid table. Then we'll describe it again.

In [ ]:
dataset['raw_citric acid'] = dataset['raw_citric acid'].str.replace('-', '0')
dataset['citric acid'] = pd.to_numeric(dataset["raw_citric acid"], errors='coerce')
simple_describe(dataset['citric acid'])

Citric acid seems clean!

<h4>Density</h4>

Most density values are formatted like proper floats, so let's have pandas turn them into that, and then describe it. This time, the simple describe won't quite cut it.

In [ ]:
dataset["density"]

In [ ]:
dataset["density"] = pd.to_numeric(dataset["density"], errors='coerce')
dataset["density"].describe()

From the description, we see that almost all values (the mininum, and all quartiles) fall around the 0.99 range. Let's examine the spread of data further with the help of a frequency graph.

In [ ]:
plt.hist(dataset["density"], bins=50)  # TODO(m-jeu): Use OOP API and make clearer
plt.gca().set(title="Density frequency", xlabel="Density", ylabel="Frequency")
plt.show()

As we can see, most measurements fall around the '0.99' mark. Though some fall around the 10, and a couple fall around the 100 mark. We will assume that this is a notation error, where the dots weren't placed right, which can fairly easily be accounted for. If we assume that all measurements are supposed to be around '1', because it wouldn't be realistic for wines to have a 10x or 100x difference in density with others, we can correct for the mistake by recursively dividing the number by 10, until a more sensible measurement is reached.

In [ ]:
def divide_until_at(num: float,
                    target: float,
                    divide_by: float,
                    delta: float,
                    max_recursions: int = 0) -> float:
    """Recursively divide a number until it's either within a certain range of a target number,
    or the max number of recursions is reached.

    Args:
        num: input number to divide.
        target: target number to get within a certain range of.
        divide_by: number to divide input number by.
        delta: amount of distance the input number could be from the target number to be considered a success.
        max_recursion: amount of recursion after which the function should give up.

    Returns:
        num divided by divide_by a certain amount of times, so that it's within delta of target.
        amount of times num is divided can also be 0.
        None if max_recursions gets violated."""
    if max_recursions < 0:
        return None
    if abs(num - target) < delta:
        return num
    return divide_until_at((num / divide_by), target, divide_by, delta, (max_recursions - 1))

dataset["density"] = dataset["density"].apply(lambda x: divide_until_at(x, 1, 10, 0.1, 2))
dataset["density"].describe()

In [ ]:
simple_describe(dataset["density"], "Density")

The weird numbers have been taken care of. A single not-a-number entry remains. Let's have a look:

In [ ]:
dataset.loc[dataset["density"].isnull()][['id', 'title', 'density', 'raw_density']]

A single wine has a density of '.', which we can't really use for anything. To keep the data pure, let's drop it.

In [ ]:
dataset = dataset[dataset["density"].notna()]
simple_describe(dataset["density"], "Density")

Density is clean!

<h4>Alcohol</h4>

Most alcohol percentages are formatted like proper floats, and can be converted automatically. This allows us to analyse the actual numbers as well:

In [ ]:
dataset["alcohol"] = pd.to_numeric(dataset["alcohol"], errors='coerce')  # No OOP interface.
simple_describe(dataset["alcohol"], "Alcohol")

The alcohol-percentages we have been able to extract from the String objects seem to fall within the boundaries of what could reasonably be considered 'wine'. We are still left with 10 not-a-numbers entries where pandas wasn't able to convert the strings to numbers. Let's have a look at them.

In [ ]:
dataset.loc[dataset["alcohol"].isnull()][['id', 'title', 'designation', 'alcohol']]  # TODO(m-jeu): This can probably be made prettier

Considering only 10 wines are missing data, we can probably look these up by hand. The percentages are based on these sources:

<a href="https://www.internationalwinechallenge.com/canopy/beverage_details?wid=44074">1344. </a><a href="https://www.winemag.com/buying-guide/companhia-das-quintas-2009-quinta-da-fronteira-seleccao-do-enologo-red-douro/">1345. </a><a href="https://www.winemag.com/buying-guide/quinta-das-bandeiras-2010-passagem-reserva-red-douro/">1620. </a><a href="https://www.nataliemaclean.com/wine-reviews/quinta-da-rede-reserva-2015/317025">1670. </a><a href="https://www.winemag.com/buying-guide/terra-silvestre-2014-grande-reserva-red-tejo-portuguese-red/">1764. </a><a href="https://www.vivino.com/NL/en/adega-cooperativa-de-borba-alentejo-reserva-tinto/w/1235048?year=2013">1765. </a><a href="https://www.winemag.com/buying-guide/adega-cooperativa-cartaxo-2012-bridao-classico-red-tejo/">1766. </a><a href="https://www.winemag.com/buying-guide/quinta-da-lagoalva-de-cima-2015-lagoalva-barrel-selection-red-tejo/">1794. </a><a href="https://www.winemag.com/buying-guide/quinta-do-casal-branco-2014-lobo-e-falcao-reserva-red-tejo/">1795. </a><a href="https://www.wine-searcher.com/find/ramos+pinto+duas+quinta+rsrv+douro+portugal/2006/netherlands#t2">2028. </a>

In [ ]:
# Assignments use numeric row index, and not 'id' column. Might break when working on the id column.
dataset.loc[1344, "alcohol"] = 14.0
dataset.loc[1345, "alcohol"] = 14.5
dataset.loc[1620, "alcohol"] = 14.0
dataset.loc[1670, "alcohol"] = 13.5
dataset.loc[1764, "alcohol"] = 13.5
dataset.loc[1765, "alcohol"] = 14.0
dataset.loc[1766, "alcohol"] = 14.0
dataset.loc[1794, "alcohol"] = 14.0
dataset.loc[1795, "alcohol"] = 14.0
dataset.loc[2028, "alcohol"] = 14.0

Lets' have a look how many nan entries are left in the alcohol column:

In [ ]:
dataset["alcohol"].isnull().sum()

Alcohol is clean!

Having converted and cleaned up all tables in the dataset that Pandas interpreted as strings, all that remains is to clean up the remaining columns.

<h4>Designation</h4>

Designation contains what 'category' a wine is. This should be a categorial variable.

In [ ]:
dataset["designation"] = dataset["designation"].astype("category")

Done!

<h4>Points</h4>

Points contains discrete values that represent review-points. Data-exploration showed that this contained no unexpected-values, and is stored as an integer. So this needs no extra work.

<h4>Price</h4>

Data exploration showed that price is treated as an integer, which makes sense. The histogram in data exploration showed that most measurements fall within the 0-100 range, with some in the 100-200 range. Notably, the x-axis on the histogram stretches all the way to around 400, with no measurements visible (probably because the frequency is so low). Let's see whether there's some hard to see measurements there.


In [ ]:
dataset[dataset["price"] > 300]

Let's look up these wines by hand just to be sure these are accurate.
<a href="https://www.vivino.com/US/en/casa-ferreirinha-barca-velha-douro/w/75975?year=2008">Wine 833</a> is 638.48 dollars at the time of writing.
<a href="https://www.wine-searcher.com/find/casa+ferreirinha+barca+velha+douro+portugal/2004/europe">Wine 1078</a> is 633 dollars at the time of writing.

It makes sense that these wines have gotten more expensive since the dataset was created. We'll take this as enough evidence to take these prices as fact, and not change them.

<h4>Province</h4>

Province contains strings, and should be categorical. Let's change them to categories, and then have a look at the values.


In [ ]:
dataset["province"] = dataset["province"].astype("category")
dataset["province"].value_counts()

There are some probable mistakes in there. Most notably, different language versions of the samen regions, and 'wine regions' being used instead of provinces. Let's go by the most important ones:
<ol>
<li>Alentejano isn't a province, and not some other place in portugal as well. We will assume that this is an an English version of Alentejo, which is an actual province.</li>
<li><a href="https://www.portevinho.be/tejo">Tejo is another word for Ribatejo</a></li>
<li><a href="https://en.wikipedia.org/wiki/Bairrada_DOC">Bairrada is a part of the Beira Litoral province</a></li>
<li>Dão is displayed incorrectly by jupiter, but <a href="https://www.quintadoriodao.com/ned/out/dao-wijngaarden-portugal.html">is a wine region in Beira Alta province</a></li>
<li>Península de Setúbal is apart of the Estremadura province</li>
<li>Beira Interior is apart of beira alta</li>
</ol>

We could continue this for a while, but that would take many hours of labor. We'll assume that the errors in the smaller categories won't influence any potential models too much, and possibly fix all the problems in the future if needed.

For now, we'll take all categories with less then 50 members, and and them to a new 'Other' class.

<i><a href="https://althistory.fandom.com/wiki/Provinces_of_Portugal_(Twilight_of_a_New_Era)">Source for provinces</a></i>




In [ ]:
dataset["province"] = dataset["province"].replace({
    "Alentejano": "Alentejo",
    "Tejo": "Ribatejo",
    "Bairrada": "Beira Litoral",  # FIXME(m-jeu): Check wether this is correct
    "D�o": "Beira Alta",
    "Pen�nsula de Set�bal": "Estremadura",
    "Beira Interior": "Beira Alta"
})

province_freqs = dataset["province"].value_counts()
to_replace = province_freqs[province_freqs < 50].index.values
dataset["province"] = dataset["province"].replace(to_replace, "Other")

dataset["province"].value_counts()

That's good for now.

<h4>Winery</h4>

Winery should be a categorical variable, so let's turn it into that.



In [ ]:
dataset["winery"] = dataset["winery"].astype("category")
dataset["winery"].value_counts()

There are no obvious problems with it.

<h4>Fixed acidity</h4>

From data exploration we know Fixed acidity is a continuous variable, which is stored as Float64, without problematic outliers. There is not much to change.

<h4>Volatile acidity</h4>

From data exploration we know Volatile acidity is a continuous variable, which is stored as Float64, without problematic outliers. There is not much to change.

<h4>Residual sugar</h4>

From data exploration we know Residual sugar is a continuous variable, which is stored as Float64, without problematic outliers. There is not much to change.

<h4>Chlorides</h4>

From data exploration we know Chlorides is a continuous variable, which is stored as Float64, without problematic outliers. There is not much to change.

<h4>Free sulfur dioxide</h4>

From data exploration we know Free sulfur dioxide is a continuous variable, which is stored as Float64, without problematic outliers. There is not much to change.

<h4>Total sulfur dioxide</h4>

From data exploration we know Total sulfur dioxide is a continuous variable, which is stored as Float64, without problematic outliers. There is not much to change.

<h4>pH</h4>

From data exploration we know pH is a continuous variable, which is stored as Float64, without problematic outliers. There is not much to change.

<h4>Sulphates</h4>

From data exploration we know Sulphates is a continuous variable, which is stored as Float64, without problematic outliers. There is not much to change.

In [ ]:
dataset.dtypes

<h3>Dropping unneeded columns</h3>

We drop columns description, taster_name and id, because they are not useful for later models, and hold no sway over our target variable, points. We also drop raw_citric acid, raw_density and raw_alcohol because they are no longer needed in data preparation.

Even though it's not a feature or target, we won't drop title, because we still need it later to help draw conclusions from the data.

Leaving us with a dataset containing clean target and feature variables.


In [ ]:
dataset = dataset.drop(['country', 'variety', 'description', 'taster_name', 'id', 'raw_citric acid', 'raw_density', 'raw_alcohol'], axis=1)

In [ ]:
dataset.head(5)

<h3>Post-Preperation evaluation</h3>

Having made sure all the data is clean, and stored in the right variable type, we can evaluate any possible (linear) correlations once more:

In [ ]:
correlation_matrix(dataset)

A next step in our research process is finding an external dataset to complement our existing one with, to help construct our models later in the process. Our original dataset came without source, or explanation of it's origin, but while looking for other datasets to add, we ran into <a href="https://www.kaggle.com/zynicide/wine-reviews">this dataset</a> and <a href="https://archive.ics.uci.edu/ml/datasets/wine+quality">this dataset</a>.
The second dataset seems to correspond to the chemical properties contained in our original dataset. The first dataset seems to correspond to everything else in the original.

We will start referring to the first (new) dataset as the review dataset, the second one as the chemical dataset, and our original dataset as the HU dataset.

The chemical dataset is subdivided into 2 files, for red and white wine respectively.
Let's load them in and have a look:


In [ ]:
chem_dataset_red = pd.read_csv("datasets/winequality-red.csv", sep=";")
chem_dataset_white = pd.read_csv("datasets/winequality-white.csv", sep=";")
review_dataset = pd.read_csv("datasets/winemag-data-130k-v2.csv")

In [ ]:
chem_dataset_red.head(2)

In [ ]:
chem_dataset_white.head(2)

In [ ]:
review_dataset.head(2)

Looking through the chemical dataset, it turns out to be <i>anonymised</i>, which is odd, because there are no discernible features to match the review dataset and the chemical dataset on, to create the HU dataset.

The sources for the 2 datasets are also wildly different. The wine dataset was scraped from a website called by a Kaggle user WineEnthusiast in 2017, the chemical dataset has as source:

Paulo Cortez, University of Minho, Guimarães, Portugal, http://www3.dsi.uminho.pt/pcortez
A. Cerdeira, F. Almeida, T. Matos and J. Reis, Viticulture Commission of the Vinho Verde Region(CVRVV), Porto, Portugal @2009

A paper by a Portuguese professor into data mining from 2009.


Looking back at the earlier correlation matrix with the realisation that the HU dataset might have been created from two separate datasets shows that there are no strong (linear) correlations between attributes from the chemical dataset, and from the review dataset, the strongest one is between the 'alcohol' and 'points' attributes, with a Pearson correlation coefficient of 0.47, which is mediocre at best.

Interestingly, this correlation appears to be very close to the correlation between alcohol and 'quality' in both chemical datasets. In those, quality is another metric for a score that someone gave the wine:

In [ ]:
print("Alcohol/Quality correlation in red chemical dataset:")
np.corrcoef(chem_dataset_red['alcohol'], chem_dataset_red['quality'])

In [ ]:
print("Alcohol/Quality correlation in white chemical dataset:")
np.corrcoef(chem_dataset_white['alcohol'], chem_dataset_white['quality'])

This could mean that the datasets were matched on a correlation between the two. In case this is based on a more complicated polynomial/logarithmic correlation, let's scatter the two in the HU dataset to examine them visually:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.set_title("Points vs. Alcohol Percentage in HU dataset.")
ax.set_xlabel("Alcohol percentage")
ax.set_ylabel("Points")
ax.scatter(dataset["alcohol"], dataset["points"], s=15)
plt.show()

There are no obvious complex correlations, because the data appears to be way too scattered to draw any conclusions from. This also means that basing a merge of 2 datasets on this would be questionable at best.

To verify our dataset-merge hypothesis even further, we also decided to manually look up records from both the left and right dataset manually, and check whether they match. The easiest way to do this seems to be to look up a wine review (allegedly from the review dataset), and it's corresponding alcohol percentage:

In [ ]:
dataset[["title", "alcohol"]].head(10)

Quinta dos Avidagos 2011 Avidagos Red (Douro)
According to the dataset: 9.7%
According to <a href="https://www.wine-searcher.com/find/quinta+dos+avidagos+tinto+douro+portugal/2011/netherlands#t2">the internet</a>: Between 13 and 14 %

Casa Santa Vit�ria 2013 Grande Reserva Tinto Red (Alentejano)
According to the dataset: 9.6%.
According to <a href="https://www.wine-searcher.com/find/casa+de+santa+vittoria+grand+rsrv+regional+alentejano+alentejo+portugal/2013#t2">the internet.</a>: 14%.

Luis Duarte 2013 Monte de Carrapatelo Colheita Seleccionada Tinto Red (Alentejano) (index 5):
According to the dataset: 12.3%
According to <a href="https://www.specialtycellars.com/wp-content/uploads/2020/08/Monte-De-Carrapatelo-Tinto-2013.pdf">the internet:</a> 14.5%


<h3>Conclusion</h3>

This all leads us to conclude that the HU dataset has probably been merged from 2 different datasets in at best an undocumented way, and at worst an illogical way. Combining this with the fact that there is no discernible source for the dataset, has lead us to decide that in the spirit of the scientific process with regard to transparency, we can't draw conclusions from the HU dataset in good faith. We will rephrase the questions asked to fit the review- and chemicaldataset and use those instead in the next iteration.